## This is where I will do the classifications for my data

* First, I need to load the data and quickly merge the dataframes...

In [157]:
import pandas as pd
import numpy as np 
import utils 
import importlib
importlib.reload(utils)


# First we need to import our data from a csv file: 
df = pd.read_csv("AutoSleep.csv", header=0)
days_df = pd.read_csv("days_of_week.csv", header=0)

df_2 = pd.read_csv("AutoSleep_part2.csv", header=0)
days_part_2 = pd.read_csv("days_test.csv", header=0)
join = [df, df_2]
join_days = [days_df, days_part_2]

df = pd.concat(join, ignore_index=True)
df = df.iloc[:-1 , :]
# df.drop(df.index[-1], axis=0, inplace=True)
days_df = pd.concat(join_days, ignore_index=True)
days_df = days_df.iloc[:-1 , :]
# days_df.drop(days_df.index[-1], axis=0, inplace=True)

for col in df:
    check_for_nan = df[col].isnull().values.any()
    if check_for_nan == True:
        # print (col, check_for_nan, df[col].isnull().count())
        df.drop(col, axis=1, inplace=True)

df.drop(['ISO8601', 'toDate', 'inBed',
       'awake', 'fellAsleepIn', 'sessions', 'asleepAvg7', 'efficiencyAvg7', 
       'qualityAvg7', 'sleepBPMAvg7', 'wakingBPMAvg7','hrvAvg7'], axis=1, inplace=True)


merged_df = days_df.merge(df, on='fromDate')
merged_df = utils.clean_sleep(df, 'asleep')


df['efficiency'] = df['efficiency'].astype('int')
df.columns

Index(['fromDate', 'bedtime', 'waketime', 'asleep', 'efficiency', 'quality',
       'sleepBPM', 'wakingBPM', 'hrv'],
      dtype='object')

### KNN Classifier 

In [158]:
# distribution of the data 
attribute = merged_df['efficiency']
attribute.describe()

count     60.000000
mean      93.916667
std       10.312559
min       55.000000
25%       93.750000
50%       98.000000
75%      100.000000
max      100.000000
Name: efficiency, dtype: float64

In [161]:
# Apply scaling 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

le = LabelEncoder()
df_encoded = merged_df.copy()
clean_merged = df_encoded.drop(['fromDate', 'efficiency'], axis=1)
for col in clean_merged:
    df_encoded[col] = le.fit_transform(clean_merged[col])
df_encoded

X = df_encoded.drop(['fromDate','efficiency'], axis=1)
y = df_encoded['efficiency']

scaler = MinMaxScaler()
X = scaler.fit_transform(X) 

X_train = X[0:54]
y_train = y.drop([54, 55, 56, 57, 58, 59], axis=0)
X_test = X[54:60]
y_actual = y.iloc[54:60]

<class 'numpy.ndarray'>


In [169]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold 


# train/build/fit
knn_clf = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
knn_clf.fit(X_train, y_train)

# # predict
y_predicted_knn = knn_clf.predict(X_test)
print("the predicted values are:", y_predicted_knn)

print("the actual values are:", y_actual.to_list())
knn_accuracy = accuracy_score(y_actual, y_predicted_knn)
print(knn_accuracy)

the predicted values are: [ 99  99  87  87  87 100]
the actual values are: [99, 98, 97, 100, 95, 65]
0.16666666666666666


In [172]:
from sklearn import tree 

tree_clf = tree.DecisionTreeClassifier(random_state=0)

#train the model
tree_clf.fit(X_train, y_train)
tree_clf.score(X_train, y_train)
y_predicted = tree_clf.predict(X_test)
print(y_predicted, y_actual)
accuracy = accuracy_score(y_actual, y_predicted)
print(accuracy)

[ 76  76  99  99 100  73] 54     99
55     98
56     97
57    100
58     95
59     65
Name: efficiency, dtype: int32
0.0


In [ ]:
import matplotlib.pyplot as plt

